In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import trange
from google.colab import files
from google.colab import drive
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
import os

In [2]:
try:
  drive.mount('/content/gdrive', force_remount=True)

  FOLDERNAME = 'Proyecto_Focaccia'
  %cd /content/gdrive/My\ Drive/ML/$FOLDERNAME
except ImportError:
  pass

Mounted at /content/gdrive
/content/gdrive/My Drive/ML/Proyecto_Focaccia


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#importacion de las datas
data = pd.read_csv('sentences.csv',
                            sep='\t', 
                            encoding='utf8', 
                            index_col=0,
                            names=['lang','text'])

Pequeño test para entender de que se trata

In [ ]:
def n_gramizer(word, n=2):
  list_n_gram = []
  word += "_"
  word = "_" + word

  if(len(word) < n):
    list_n_gram.append(word)
  else:
    for i in range(len(word) + 1 - n):
      list_n_gram.append(word[i:i+n])
  
  return list_n_gram


In [ ]:
test = "Italie"
test_ngram = n_gramizer(test, n=3)
print(test_ngram)

['_It', 'Ita', 'tal', 'ali', 'lie', 'ie_']


## I) Pequeño test con dos idiomas differentes muy bonitos : Español  y Frances


In [ ]:
lang = ['fra', 'spa']
mask = data['lang'].isin(lang)
mask_fra = data['lang'] == 'fra'
mask_spa = data['lang'] == 'spa'

In [ ]:
data = data[mask]
data_fr = data[mask_fra]
data_spa = data[mask_spa]

Tenemos que tener lo mismo numero di data para el francés y el español también y también tenemos que sea muy bien mezclado 

In [ ]:
#parameters
N_lang = 1000
N_train = 1000 

n_gram = 3
K_most_common = 200


In [ ]:
data_fr = data_fr.sample(N_lang)
data_spa = data_spa.sample(N_lang)

data = data_fr.append(data_spa)
data = data.sample(frac=1)

In [ ]:
data_fr

In [ ]:
len(data_spa)
len(data_fr)

1000

Me parece muy utíl de ulitizar esta función, pruebo de hacerlo sin estar seguro que las n_gram sean unicos per algun idioma.

In [ ]:
data[ data['lang'] == 'fra']['text']

In [ ]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(n_gram, n_gram) , max_features = K_most_common)

X_fr = vectorizer.fit_transform(data[ data['lang'] == 'fra']['text'])
n_gram_fr = vectorizer.get_feature_names_out()
X_spa = vectorizer.fit_transform(data[ data['lang'] == 'spa']['text'])
n_gram_spa = vectorizer.get_feature_names_out()

n_gram_total = np.setxor1d(n_gram_fr, n_gram_spa)

In [ ]:
n_gram_total

In [ ]:
# es para hacer la feature_matrix final, la vectorizer funcion quiere un diccionario
vocab_n_gram = dict()
for i in range(n_gram_total.size):
  vocab_n_gram[n_gram_total[i]] = i

In [ ]:
vectorizer_final = CountVectorizer(analyzer='char', ngram_range=(n_gram, n_gram), vocabulary = vocab_n_gram)
X_tot = vectorizer_final.fit_transform(data['text'])
feature_matrix = pd.DataFrame(data = X_tot.toarray(), columns = vectorizer_final.get_feature_names_out())

Scaling de la feature matrix

In [ ]:
feature_matrix = (feature_matrix - feature_matrix.min()) / (feature_matrix.max()-feature_matrix.min())

In [ ]:
feature_matrix

Encoding de la target, 0 por 'spa', 1 por 'fra'

In [ ]:
y_target = torch.zeros([len(feature_matrix)], dtype=torch.float64)
y_target = y_target + int(1) * (data['lang'].values == 'fra')
#y_target = nn.functional.one_hot(y_target, num_classes=2)

Ahora podemos empezar de utilizar deepl para predecir los idiomas. 

In [ ]:
print(feature_matrix.values)

In [ ]:
'''
import torch.utils.data as data_utils

# Creating np arrays
target = df['Target'].values
features = df.drop('Target', axis=1).values

# Passing to DataLoader
train = data_utils.TensorDataset(features, target)
train_loader = data_utils.DataLoader(train, batch_size=10, shuffle=True)

'''
train_matrix = torch.tensor(feature_matrix.iloc[0:N_train].values, dtype=torch.float64)
val_matrix = torch.tensor(feature_matrix.iloc[N_train:len(feature_matrix)].values, dtype=torch.float64)

# Passing to DataLoader
train_data_set = data_utils.TensorDataset(train_matrix, y_target[0:N_train])
val_data_set = data_utils.TensorDataset(val_matrix, y_target[N_train:len(feature_matrix)])
loader_train = data_utils.DataLoader(train_data_set, batch_size=10)
loader_val = data_utils.DataLoader(val_data_set, batch_size=10)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred, y.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Pruebo el model del articolo

In [ ]:
model = nn.Sequential(
    nn.Linear(n_gram_total.size,500),
    nn.ReLU(inplace=True),
    nn.Linear(500,500),
    nn.ReLU(inplace=True),
    nn.Linear(500,250),
    nn.ReLU(inplace=True),
    nn.Linear(250,2),
    nn.ReLU(inplace=True),
    nn.Softmax(dim=1)
)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss = nn.CrossEntropyLoss()


In [ ]:
train(loader_train, model, loss, optimizer)
test(loader_val, model, loss)

loss: 0.696215  [    0/ 1000]
loss: 0.686567  [  100/ 1000]
loss: 0.601622  [  200/ 1000]
loss: 0.631212  [  300/ 1000]
loss: 0.521622  [  400/ 1000]
loss: 0.412702  [  500/ 1000]
loss: 0.489608  [  600/ 1000]
loss: 0.465241  [  700/ 1000]
loss: 0.468154  [  800/ 1000]
loss: 0.557216  [  900/ 1000]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.514808 



Muy Bien no ?

# II) Importacion del data_set con los dialectos español

He hecho un data_set por dialecto, con un diccionario.

In [4]:
diferentes_dialectos = ["argentina", "chile", "colombia", "espana", "guatemala", "mexico", "panama", "venezuela"]

In [5]:
all_dialectos_files = os.listdir("NewsCorpus/") # hay todos los nomes de los .txt

In [6]:
data_dialectos_list = []

Hay problamente una cosa mas inteligente para hacerlo

In [7]:
for name_file in all_dialectos_files:
  name_dir_y_file = "NewsCorpus/" + name_file
  texto_file = open(name_dir_y_file, "r")
  data_articulo = texto_file.read()
  texto_file.close()
  data_dialectos_list.append(data_articulo)
  

In [8]:
data_dialectos = pd.DataFrame(data_dialectos_list, columns = ["text"])

In [ ]:
data_dialectos

Incorporación de los nombres de dialecto

In [9]:
argentina = np.full(449, "argentina")
chile = np.full(375, "chile")
colombia = np.full(929, "colombia")
espana = np.full(908, "espana")
guatemala = np.full(598, "guatemala")
mexico = np.full(682, "mexico")
panama = np.full(418, "panama")
venezuela = np.full(828, "venezuela")

columnas_todos_dialectos = np.concatenate((argentina,chile,colombia,espana,guatemala,mexico,panama,venezuela))

Verificacion 

In [10]:
columnas_todos_dialectos.size == data_dialectos.size

True

In [11]:
data_dialectos["lang"] = columnas_todos_dialectos

In [12]:
data_dialectos

,text,lang
0,"Anastacio Ruiz De León, exmiembro de la junta ...",argentina
1,El Sistema Nacional de Protección Civil (Sinap...,argentina
2,El Juzgado Decimosexto de lo Penal negó cambia...,argentina
3,La Fiscalía Primera Anticorrupción reanudó la...,argentina
4,"Ricardo Arango Pezet, exdirectivo de la Caja d...",argentina
...,...,...
5182,"""Esto no es Suiza” decía, hace apenas unos mes...",venezuela
5183,Le quedan 114 días simbólicos de su doble inqu...,venezuela
5184,Entró puntual al bar de la cita. De antemano s...,venezuela
5185,El pulseo lleva diez meses y lo encarrila un f...,venezuela


# III) Primeras pruebas con esto data_set

Con solo dos dialectos

In [13]:
dos_dialectos = ["argentina", "mexico"]
mask_dos_dialectos = data_dialectos["lang"].isin(dos_dialectos)
data_dialectos_dos = data_dialectos[mask_dos_dialectos]

In [14]:
data_dialectos_dos

,text,lang
0,"Anastacio Ruiz De León, exmiembro de la junta ...",argentina
1,El Sistema Nacional de Protección Civil (Sinap...,argentina
2,El Juzgado Decimosexto de lo Penal negó cambia...,argentina
3,La Fiscalía Primera Anticorrupción reanudó la...,argentina
4,"Ricardo Arango Pezet, exdirectivo de la Caja d...",argentina
...,...,...
3936,El cineasta estadounidense Jeff Nichols es de ...,mexico
3937,Podemos celebra este sábado un Consejo Ciudada...,mexico
3938,Dio un giro a la legislatura actual de Castill...,mexico
3939,Íñigo Errejón ha reaparecido ante los medios d...,mexico


He hecho lo mismo che he habia hecho con el primero data_set

In [15]:
n_gram = 3
K_most_common = 500

N_train = 500

In [16]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(n_gram, n_gram) , max_features = K_most_common)

X_argentina = vectorizer.fit_transform(data_dialectos_dos[ data_dialectos_dos['lang'] == 'argentina']['text'])
n_gram_argentina = vectorizer.get_feature_names_out()
X_mexico = vectorizer.fit_transform(data_dialectos_dos[ data_dialectos_dos['lang'] == 'mexico']['text'])
n_gram_mexico = vectorizer.get_feature_names_out()

n_gram_total = np.setxor1d(n_gram_argentina, n_gram_mexico)

In [17]:
n_gram_argentina

array([' a ', ' ac', ' ad', ' al', ' an', ' ap', ' as', ' añ', ' ba',
       ' ca', ' ci', ' co', ' cr', ' cu', ' de', ' di', ' do', ' el',
       ' em', ' en', ' es', ' ex', ' fa', ' fi', ' fu', ' ha', ' ho',
       ' im', ' in', ' ju', ' la', ' le', ' ll', ' lo', ' ma', ' me',
       ' mi', ' mo', ' mu', ' má', ' na', ' ni', ' no', ' o ', ' pa',
       ' pe', ' po', ' pr', ' pu', ' qu', ' re', ' sa', ' se', ' si',
       ' so', ' su', ' ta', ' te', ' ti', ' to', ' tr', ' un', ' ve',
       ' vi', ' y ', ', a', ', c', ', d', ', e', ', l', ', p', ', s',
       '. e', '. l', '. p', '. s', 'a a', 'a c', 'a d', 'a e', 'a f',
       'a h', 'a i', 'a l', 'a m', 'a n', 'a o', 'a p', 'a q', 'a r',
       'a s', 'a t', 'a u', 'a v', 'a y', 'a, ', 'a. ', 'aba', 'abl',
       'ace', 'aci', 'act', 'ad ', 'ada', 'ade', 'ado', 'al ', 'ale',
       'ali', 'amb', 'ame', 'ami', 'an ', 'ana', 'anc', 'and', 'ano',
       'ant', 'ar ', 'ara', 'are', 'ari', 'aro', 'art', 'as ', 'as,',
       'asa', 'ast',

In [18]:
n_gram_mexico

array([' a ', ' ac', ' ad', ' al', ' an', ' ap', ' as', ' au', ' añ',
       ' ba', ' ca', ' ci', ' co', ' cr', ' cu', ' de', ' di', ' do',
       ' el', ' em', ' en', ' es', ' ex', ' fa', ' fi', ' fu', ' gr',
       ' ha', ' in', ' ju', ' la', ' le', ' ll', ' lo', ' ma', ' me',
       ' mi', ' mo', ' mu', ' má', ' ni', ' no', ' o ', ' ot', ' pa',
       ' pe', ' po', ' pr', ' pu', ' qu', ' re', ' sa', ' se', ' si',
       ' so', ' su', ' ta', ' te', ' ti', ' to', ' tr', ' un', ' va',
       ' ve', ' vi', ' y ', ', a', ', c', ', d', ', e', ', l', ', p',
       ', q', ', s', '. a', '. e', '. l', '. s', 'a a', 'a c', 'a d',
       'a e', 'a f', 'a h', 'a i', 'a l', 'a m', 'a n', 'a o', 'a p',
       'a q', 'a r', 'a s', 'a t', 'a u', 'a v', 'a y', 'a, ', 'a. ',
       'aba', 'ace', 'aci', 'act', 'ad ', 'ada', 'ade', 'ado', 'al ',
       'ale', 'ali', 'amb', 'ame', 'ami', 'an ', 'ana', 'anc', 'and',
       'ano', 'ant', 'ar ', 'ara', 'are', 'ari', 'arr', 'art', 'as ',
       'as,', 'asa',

In [19]:
n_gram_total

array([' au', ' gr', ' ho', ' im', ' na', ' ot', ' va', ', q', '. a',
       '. p', 'abl', 'aro', 'arr', 'ast', 'ata', 'ati', 'ber', 'cam',
       'ce ', 'cid', 'col', 'cre', 'cul', 'd d', 'e o', 'eda', 'ede',
       'ela', 'ell', 'eta', 'exp', 'hab', 'ill', 'imp', 'ini', 'ino',
       'isi', 'ism', 'ito', 'ivo', 'leg', 'liz', 'mad', 'me ', 'med',
       'mpl', 'n q', 'nac', 'nam', 'nar', 'nas', 'ne ', 'nez', 'ola',
       'otr', 'pan', 'pas', 'pla', 'pli', 'pol', 'ral', 'rea', 'rso',
       's f', 's o', 's v', 'sió', 'stá', 'tur', 'uch', 'ued', 'uel',
       'ula', 'uro', 'vis', 'ños'], dtype=object)

In [20]:
# es para hacer la feature_matrix final, la vectorizer funcion quiere un diccionario
vocab_n_gram = dict()
for i in range(n_gram_total.size):
  vocab_n_gram[n_gram_total[i]] = i

In [21]:
vectorizer_final = CountVectorizer(analyzer='char', ngram_range=(n_gram, n_gram), vocabulary = vocab_n_gram)
X_tot = vectorizer_final.fit_transform(data_dialectos_dos['text'])
feature_matrix = pd.DataFrame(data = X_tot.toarray(), columns = vectorizer_final.get_feature_names_out())

Scaling de la feature matrix

In [22]:
feature_matrix = (feature_matrix - feature_matrix.min()) / (feature_matrix.max()-feature_matrix.min())

In [23]:
feature_matrix

,au,gr,ho,im,na,ot,va,", q",. a,. p,...,sió,stá,tur,uch,ued,uel,ula,uro,vis,ños
0,0.0000,0.000000,0.0000,0.166667,0.000000,0.086957,0.052632,0.166667,0.0000,0.000000,...,0.1250,0.05,0.000000,0.000000,0.034483,0.08,0.00,0.000000,0.071429,0.000000
1,0.1250,0.000000,0.0000,0.000000,0.047619,0.000000,0.105263,0.083333,0.2500,0.083333,...,0.0625,0.15,0.028571,0.000000,0.068966,0.12,0.00,0.000000,0.285714,0.000000
2,0.1250,0.000000,0.0000,0.166667,0.095238,0.086957,0.000000,0.083333,0.0000,0.000000,...,0.0625,0.10,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.071429,0.000000
3,0.0000,0.000000,0.0000,0.000000,0.047619,0.043478,0.000000,0.250000,0.0625,0.041667,...,0.0000,0.00,0.000000,0.000000,0.000000,0.00,0.04,0.000000,0.071429,0.000000
4,0.2500,0.000000,0.0000,0.611111,0.047619,0.173913,0.052632,0.166667,0.0625,0.000000,...,0.0000,0.05,0.085714,0.043478,0.034483,0.04,0.08,0.000000,0.142857,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,0.0625,0.107143,0.0625,0.055556,0.190476,0.173913,0.052632,0.166667,0.1875,0.083333,...,0.0000,0.05,0.142857,0.043478,0.103448,0.04,0.24,0.038462,0.071429,0.076923
1127,0.1875,0.071429,0.1250,0.166667,0.095238,0.086957,0.263158,0.166667,0.1875,0.291667,...,0.0625,0.25,0.057143,0.173913,0.068966,0.08,0.08,0.038462,0.000000,0.000000
1128,0.3125,0.107143,0.3125,0.222222,0.428571,0.217391,0.578947,0.500000,0.1875,0.375000,...,0.5625,0.40,0.200000,0.391304,0.344828,0.08,0.04,0.038462,0.285714,0.038462
1129,0.0625,0.000000,0.1250,0.000000,0.000000,0.130435,0.000000,0.083333,0.0000,0.083333,...,0.1875,0.05,0.028571,0.043478,0.068966,0.00,0.04,0.000000,0.142857,0.038462


Encoding de la target, 1 por 'mexico', 0 por 'argentina'

In [24]:
y_target = torch.zeros([len(feature_matrix)], dtype=torch.float64)
y_target = y_target + int(1) * (data_dialectos_dos['lang'].values == 'mexico')
#y_target = nn.functional.one_hot(y_target, num_classes=2)

Ahora podemos empezar de utilizar deepl para predecir los idiomas. 

In [ ]:
y_target

tensor([0., 0., 0.,  ..., 1., 1., 1.], dtype=torch.float64)

In [ ]:
feature_matrix.values

In [ ]:
'''
import torch.utils.data as data_utils

# Creating np arrays
target = df['Target'].values
features = df.drop('Target', axis=1).values

# Passing to DataLoader
train = data_utils.TensorDataset(features, target)
train_loader = data_utils.DataLoader(train, batch_size=10, shuffle=True)

'''
train_matrix = torch.tensor(feature_matrix.iloc[0:N_train].values, dtype=torch.float64)
val_matrix = torch.tensor(feature_matrix.iloc[N_train:len(feature_matrix)].values, dtype=torch.float64)

# Passing to DataLoader
train_data_set = data_utils.TensorDataset(train_matrix, y_target[0:N_train])
val_data_set = data_utils.TensorDataset(val_matrix, y_target[N_train:len(feature_matrix)])
loader_train = data_utils.DataLoader(train_data_set, batch_size=10)
loader_val = data_utils.DataLoader(val_data_set, batch_size=10)

In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred, y.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            print(pred)
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
model = nn.Sequential(
    nn.Linear(n_gram_total.size,500),
    nn.ReLU(inplace=True),
    nn.Linear(500,500),
    nn.ReLU(inplace=True),
    nn.Linear(500,250),
    nn.ReLU(inplace=True),
    nn.Linear(250,2),
    nn.ReLU(inplace=True),
    nn.Softmax(dim=1)
)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss = nn.CrossEntropyLoss()

In [ ]:
train(loader_train, model, loss, optimizer)
test(loader_train, model, loss)

loss: 0.690208  [    0/  500]
loss: 0.358482  [  100/  500]
loss: 0.313262  [  200/  500]
loss: 0.313262  [  300/  500]
loss: 0.313262  [  400/  500]
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.415262 



##Wtf cabron

In [ ]:
test(loader_val, model, loss)

Test Error: 
 Accuracy: 0.0%, Avg loss: 1.313262 



Nueva prueba

In [28]:
train_matrix = torch.tensor(feature_matrix.iloc[0:100].values, dtype=torch.float64)
val_matrix = torch.tensor(feature_matrix.iloc[100:110].values, dtype=torch.float64)

# Passing to DataLoader
train_data_set = data_utils.TensorDataset(train_matrix, y_target[0:100])
val_data_set = data_utils.TensorDataset(val_matrix, y_target[100:110])
loader_train = data_utils.DataLoader(train_data_set, batch_size=10)
loader_val = data_utils.DataLoader(val_data_set, batch_size=10)


In [29]:
train(loader_train, model, loss, optimizer)
test(loader_train, model, loss)

loss: 0.693147  [    0/  100]
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]])
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]])
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]])
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
     